In [66]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [48]:
# URL of Nasa 
url = 'https://mars.nasa.gov/news/'


In [49]:
# Retrieve page with the requests module
html = requests.get(url)

In [56]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html.text, 'lxml')

In [64]:
# Retrieve the latest news Title and Paragraph text
titleResults = soup.find('div', class_='content_title')
newsResults = soup.find('div', class_='rollover_description_inner')
title = titleResults.a.text.strip()
news = newsResults.text


In [96]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image 
JPL_URL = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'

from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/johnwanng/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [97]:
browser.visit(JPL_URL + 'index.html')
# Click on the FULL IMAGE button
browser.click_link_by_partial_text('FULL IMAGE')
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
imageResults = soup.find('div', class_='fancybox-inner')
# If the link contains the image then assign it to a variable called featured_image_url.
if imageResults.img:
    featureImageURL = JPL_URL + imageResults.img["src"]
print(featureImageURL)


https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


In [118]:
import pandas as pd
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
marsFactsURL = 'https://space-facts.com/mars/'
# Use Pandas to convert the data to a HTML table string.
tables = pd.read_html(marsFactsURL)
df = tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [242]:
marsHemispheresURL = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

hemisphere_image_urls = [
      {"title": "Cerberus Hemisphere Enhanced", "img_url": ""},
      {"title": "Schiaparelli Hemisphere Enhanced", "img_url": ""},
      {"title": "Syrtis Major Hemisphere Enhanced", "img_url": ""},
      {"title": "Valles Marineris Hemisphere Enhanced", "img_url": ""}]

# Loop through the dictionary
for i in hemisphere_image_urls:
    print(i['title'])
    print('before : ' + i['img_url'])
    # Visit the main site 
    browser.visit(marsHemispheresURL)
    # Click on the link of the title from the dictionary
    browser.click_link_by_partial_text(i['title'])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    # Get all li
    imageResults = soup.find_all('li')
    for image in imageResults:
     # If it is a href and its text is original then use its href link as the image url
     a = image.find('a')
     if a.text == 'Original':
        imageURL = a["href"]
        # Update the img_url 
        i.update({"img_url":imageURL})
        print('after: ' + i['img_url'])


Cerberus Hemisphere Enhanced
before : 
after: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif
Schiaparelli Hemisphere Enhanced
before : 
after: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif
Syrtis Major Hemisphere Enhanced
before : 
after: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif
Valles Marineris Hemisphere Enhanced
before : 
after: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif


In [243]:
print(hemisphere_image_urls)


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
